# Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import librosa
import pandas as pd
pd.set_option("display.max_columns",None)
import numpy as np
from tensorflow.keras.models import load_model
import pickle
import IPython
from IPython.display import display, HTML

# Model Loading

In [ ]:
model=load_model("/content/drive/MyDrive/ASE/Audio_Project_2/models/ArtificialNeuralNetwork_model_v1.h5")

In [ ]:
with open(file="/content/drive/MyDrive/ASE/Audio_Project_2/models/Scaler.pkl",mode="rb") as file:
    scaler=pickle.load(file=file)

# Data Loading

In [ ]:
class_labels=['UnAuthorised','Authorised']

In [ ]:
header = ['chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff zero_crossing_rate']
for i in range(1, 22):
    header.append('mfcc'+str(i))
print(header)

['chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'mfcc21']


In [ ]:
def data_convertion(songname):
    data_points=[]
    y, sr = librosa.load(songname, mono=True, duration=30)
    data_points.append(np.mean(librosa.feature.chroma_stft(y=y, sr=sr)))
    data_points.append(np.mean(librosa.feature.rms(y=y)))
    data_points.append(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)))
    data_points.append(np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)))
    data_points.append(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)))
    data_points.append(np.mean(librosa.feature.zero_crossing_rate(y)))
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    for e in mfcc:
        data_points.append(np.mean(e))
    return data_points

In [ ]:
os.listdir("/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs")

['ramya_open_garage.mp4', 'hemath_garage.m4a', 'newfile.amr']

***User Input***

In [ ]:
input_file=input("Enter / Paste filename")
filepath=os.path.join('/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs',input_file)
print(filepath)

Enter / Paste filenamehemath_garage.m4a
/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs/hemath_garage.m4a


In [ ]:
IPython.display.Audio(filepath)

In [ ]:
data_points=data_convertion(filepath)
df=pd.DataFrame(data=[data_points],columns=header)
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
0,0.380769,0.089294,1559.862558,1837.65017,3190.963041,0.06177,-323.49823,97.08548,9.728975,25.227251,-10.860122,-4.540321,-8.807902,-11.873154,-4.286044,6.267857,-16.878016,3.535169,-5.342598,-7.979262,-2.286026,-2.405164,-2.816684,-6.593769,-0.606481,0.660449


In [ ]:
scaled_input=scaler.transform(df.values)
df=pd.DataFrame(data=scaled_input,columns=header)
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
0,-0.296156,0.568215,-0.836953,-0.346002,-0.330808,-1.0468,0.530268,0.427188,0.519698,1.019956,-0.66981,-0.521544,-0.420338,-2.06316,0.302547,0.410841,-1.530749,0.984289,-0.555969,-0.994259,0.281417,0.378523,0.257261,-0.734546,0.396405,0.040353


# Result

In [ ]:
pred=model.predict(df.values)

class_name=np.argmax(pred[0])
label_name=class_labels[class_name]

print(f"Class Name: {class_name}")
print(f"Label Name: {label_name}")

if class_name==1:
    display(HTML("<h1> Permission granted </h1>"))
else:
    display(HTML("<h1> Permission denied </h1>"))

1/1 [==============================] - 0s 28ms/step
Class Name: 0
Label Name: UnAuthorised


---

In [ ]:
import urllib.request 

input_file=input("Enter / Paste aws link")
filepath=os.path.join('/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs','newfile.amr')
print(filepath)
urllib.request.urlretrieve(input_file, filepath)


Enter / Paste aws linkhttps://ocsimagebucket.s3.amazonaws.com/1682721768461recording.amr
/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs/newfile.amr


('/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs/newfile.amr',
 <http.client.HTTPMessage at 0x7f4901b28580>)

In [ ]:
!pip install ffmpeg

import subprocess

import ffmpeg

input_file = '/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs/newfile.amr'
output_file = '/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs/newfile.mp4'
os.remove(output_file)

subprocess.run(['ffmpeg', '-i', input_file, '-ar', '16000', output_file])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


CompletedProcess(args=['ffmpeg', '-i', '/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs/newfile.amr', '-ar', '16000', '/content/drive/MyDrive/ASE/Audio_Project_2/user_inputs/newfile.mp4'], returncode=0)

In [ ]:
IPython.display.Audio(output_file)

In [ ]:
data_points=data_convertion(output_file)
df=pd.DataFrame(data=[data_points],columns=header)
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
0,0.429729,0.050505,1016.718388,1233.598553,1962.819362,0.066331,-426.301849,133.892593,14.810082,29.176672,3.519253,2.239727,-4.17104,-5.401802,-5.219927,-4.009501,-1.301807,-8.285537,-3.08363,-5.201906,-1.422017,0.861405,-6.306606,-5.239551,-3.159421,-5.753618


In [ ]:
scaled_input=scaler.transform(df.values)
df=pd.DataFrame(data=scaled_input,columns=header)
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
0,0.746869,-1.489606,-2.89662,-3.219129,-2.608252,-0.817828,-3.428039,3.570438,0.955043,1.291004,0.447419,0.274004,0.363809,-1.077532,0.145503,-2.492496,1.380127,-1.584761,0.056327,-0.484582,0.440412,1.306308,-0.605765,-0.354744,-0.243235,-1.018766


In [ ]:
pred=model.predict(df.values)

class_name=np.argmax(pred[0])
label_name=class_labels[class_name]

print(f"Class Name: {class_name}")
print(f"Label Name: {label_name}")

if class_name==1:
    display(HTML("<h1> Permission granted </h1>"))
else:
    display(HTML("<h1> Permission denied </h1>"))

1/1 [==============================] - 0s 38ms/step
Class Name: 0
Label Name: UnAuthorised
